In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
import hopsworks

project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store()

/Users/martinbravodiaz/miniconda3/envs/imdb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-01-04 13:10:46,690 INFO: Initializing external client
2025-01-04 13:10:46,690 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-04 13:10:48,261 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559


In [3]:
tmdb_movie_fg = fs.get_feature_group(
    "tmdb_movies", 
    version=1
)
df = tmdb_movie_fg.read()

tmdb_movie_genres_fg = fs.get_feature_group(
    "tmdb_movie_genres", 
    version=1
)
df_genres = tmdb_movie_genres_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.02s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.57s) 


In [4]:
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,timestamp
0,106546,Moscow Heat,2.0,12.0,Released,2004-11-24,840201.0,89.0,0.0,tt0369738,...,"Joanna Pacula, Evgeniy Berezovskiy, Jeff Celen...",Jeff Celentano,John B. Aronson,"Alexander Nevsky, Alexander Izotov, Robert Madrid","Alexander Nevsky, Alexander Izotov",Richard John Baker,2.5,2438.0,/7WwaL21mcewnQTVAz3zOStDB4M1.jpg,2025-01-04 00:00:00+00:00
1,5526,A Killing Affair,5.2,4.0,Released,1977-09-21,0.0,100.0,0.0,tt0076266,...,"Todd Bridges, Sari Price, Dean Stockwell, Robe...",Richard C. Sarafian,Al Francis,E. Arthur Kean,"James H. Brown, David Gerber",Richard Shores,6.1,125.0,/hbb3Y27us6gx9nBpEOcMUcvh7qB.jpg,2025-01-04 00:00:00+00:00
2,194432,Top Secret,6.0,1.0,Released,1952-11-10,0.0,93.0,0.0,tt0045244,...,"Reed De Rouen, Henry Hewitt, Michael Medwin, W...",Mario Zampi,Stanley Pavey,"Michael Pertwee, Jack Davies",Mario Zampi,Stanley Black,6.3,201.0,/ss0h7JDyYMPYR2cP5fSlmpqQGD7.jpg,2025-01-04 00:00:00+00:00
3,319073,How to Change the World,7.1,33.0,Released,2014-12-12,0.0,110.0,0.0,tt4144504,...,"Robert Hunter, Bobbi Hunter, David Garrick, Pa...",Jerry Rothwell,Ben Lichty,Jerry Rothwell,"Bous De Jong, Stewart Le Marechal, Lizzie Fran...",Lesley Barber,7.5,1313.0,/994hnuZUbkAkpCZMaqasMw2cWvj.jpg,2025-01-04 00:00:00+00:00
4,68849,The Man Who Could Work Miracles,5.6,26.0,Released,1936-07-23,0.0,82.0,0.0,tt0029201,...,"George Zucco, Ernest Thesiger, Torin Thatcher,...",Lothar Mendes,Harold Rosson,"H.G. Wells, Lajos Biró",Alexander Korda,Mischa Spoliansky,6.9,1647.0,/yAfQRixZWBzQhJ7vwgCIyQhxNW9.jpg,2025-01-04 00:00:00+00:00


In [5]:
df_genres.head()

,id,timestamp,action,adventure,animation,comedy,crime,documentary,drama,family,...,history,horror,music,mystery,romance,science_fiction,tv_movie,thriller,war,western
0,270805,2025-01-04 00:00:00+00:00,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,205348,2025-01-04 00:00:00+00:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,31555,2025-01-04 00:00:00+00:00,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,302156,2025-01-04 00:00:00+00:00,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,68636,2025-01-04 00:00:00+00:00,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alanvourch/tmdb-movies-daily-updates")

In [7]:
import os
import pandas as pd

# List files in the downloaded dataset directory
files = os.listdir(path)
print("Files in dataset directory:", files)

# Read the first file
updated_df = pd.read_csv(os.path.join(path, files[0]))

# Print the first 5 rows
updated_df.head()

Files in dataset directory: ['TMDB_all_movies.csv']


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
0,2,Ariel,7.1,335.0,Released,1988-10-21,0.0,73.0,0.0,tt0094675,...,suomi,"Heikki Salomaa, Hanna Jokinen, Matti Pellonpää...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Aki Kaurismäki,NaN,7.4,8868.0,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg
1,3,Shadows in Paradise,7.3,400.0,Released,1986-10-17,0.0,74.0,0.0,tt0092149,...,"English, suomi, svenska","Teuvo Rissanen, Malla Hukkanen, Olli Varja, Ju...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Mika Kaurismäki,NaN,7.5,7646.0,/nj01hspawPof0mJmlgfjuLyJuRN.jpg
2,5,Four Rooms,5.9,2650.0,Released,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,...,English,"Unruly Julie McClean, Marc Lawrence, Jennifer ...","Robert Rodriguez, Alexandre Rockwell, Allison ...","Andrzej Sekula, Rodrigo García, Phil Parmet, G...","Robert Rodriguez, Alexandre Rockwell, Allison ...","Lawrence Bender, Alexandre Rockwell, Quentin T...",Combustible Edison,6.7,113064.0,/pyCk5JgtRZwRxnXwfrvyzukaKue.jpg
3,6,Judgment Night,6.5,333.0,Released,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,...,English,"Stephen Dorff, Everlast, Will Zahrn, Emilio Es...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Gene Levy, Marilyn Vance, Lloyd Segan",Alan Silvestri,6.6,19437.0,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg
4,8,Life in Loops (A Megacities RMX),7.5,27.0,Released,2006-01-01,0.0,80.0,42000.0,tt0825671,...,"English, हिन्दी, 日本語, Pусский, Español",NaN,Timo Novotny,Wolfgang Thaler,"Michael Glawogger, Timo Novotny","Ulrich Gehmacher, Timo Novotny",NaN,8.2,284.0,/7ln81BRnPR2wqxuITZxEciCe1lc.jpg


In [8]:
from utils import preprocess, split_genres

updated_df = preprocess(updated_df)
updated_df_genres = split_genres(updated_df)

In [9]:
updated_df_genres.head()

,id,timestamp,action,adventure,animation,comedy,crime,documentary,drama,family,...,history,horror,music,mystery,romance,science_fiction,tv_movie,thriller,war,western
2,5,2025-01-04,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,2025-01-04,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,11,2025-01-04,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,12,2025-01-04,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,13,2025-01-04,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [10]:
import pandas as pd
from utils import merge_df

df = merge_df(df, updated_df)
df_genres = merge_df(df_genres, updated_df_genres)


In [11]:
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,timestamp
0,106546,Moscow Heat,2.0,12.0,Released,2004-11-24,840201.0,89.0,0.0,tt0369738,...,"Joanna Pacula, Evgeniy Berezovskiy, Jeff Celen...",Jeff Celentano,John B. Aronson,"Alexander Nevsky, Alexander Izotov, Robert Madrid","Alexander Nevsky, Alexander Izotov",Richard John Baker,2.5,2438.0,/7WwaL21mcewnQTVAz3zOStDB4M1.jpg,2025-01-04 00:00:00+00:00
1,5526,A Killing Affair,5.2,4.0,Released,1977-09-21,0.0,100.0,0.0,tt0076266,...,"Todd Bridges, Sari Price, Dean Stockwell, Robe...",Richard C. Sarafian,Al Francis,E. Arthur Kean,"James H. Brown, David Gerber",Richard Shores,6.1,125.0,/hbb3Y27us6gx9nBpEOcMUcvh7qB.jpg,2025-01-04 00:00:00+00:00
2,194432,Top Secret,6.0,1.0,Released,1952-11-10,0.0,93.0,0.0,tt0045244,...,"Reed De Rouen, Henry Hewitt, Michael Medwin, W...",Mario Zampi,Stanley Pavey,"Michael Pertwee, Jack Davies",Mario Zampi,Stanley Black,6.3,201.0,/ss0h7JDyYMPYR2cP5fSlmpqQGD7.jpg,2025-01-04 00:00:00+00:00
3,319073,How to Change the World,7.1,33.0,Released,2014-12-12,0.0,110.0,0.0,tt4144504,...,"Robert Hunter, Bobbi Hunter, David Garrick, Pa...",Jerry Rothwell,Ben Lichty,Jerry Rothwell,"Bous De Jong, Stewart Le Marechal, Lizzie Fran...",Lesley Barber,7.5,1313.0,/994hnuZUbkAkpCZMaqasMw2cWvj.jpg,2025-01-04 00:00:00+00:00
4,68849,The Man Who Could Work Miracles,5.6,26.0,Released,1936-07-23,0.0,82.0,0.0,tt0029201,...,"George Zucco, Ernest Thesiger, Torin Thatcher,...",Lothar Mendes,Harold Rosson,"H.G. Wells, Lajos Biró",Alexander Korda,Mischa Spoliansky,6.9,1647.0,/yAfQRixZWBzQhJ7vwgCIyQhxNW9.jpg,2025-01-04 00:00:00+00:00


In [12]:
# Update to hopsworks feature store
tmdb_movie_fg.insert(df)
tmdb_movie_genres_fg.insert(df_genres)

Uploading Dataframe: 100.00% |██████████| Rows 25696/25696 | Elapsed Time: 00:16 | Remaining Time: 00:00


Launching job: tmdb_movies_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1169559/jobs/named/tmdb_movies_1_offline_fg_materialization/executions


Uploading Dataframe: 100.00% |██████████| Rows 25696/25696 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: tmdb_movie_genres_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1169559/jobs/named/tmdb_movie_genres_1_offline_fg_materialization/executions


(Job('tmdb_movie_genres_1_offline_fg_materialization', 'SPARK'), None)